# Import libraries

In [1]:
import pandas as pd
import numpy as np

# Enable interactive Matplotlib plots in the notebook
%matplotlib qt5


import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

import matplotlib.pyplot as plt
from matplotlib.widgets import Button



# Interactive Plotting Class

In [2]:
# class for interactive 2D plot

# Create a custom class to handle button clicks
class PlotUpdater:
    def __init__(self, ax, df, plot_type='2D'):
        self.ax = ax
        self.df = df
        self.plot_type = plot_type
        self.current_index = 0
        self.button_next = Button(ax, 'Next', color='lightgoldenrodyellow', hovercolor='0.975')
        self.button_prev = Button(ax, 'Previous', color='lightgoldenrodyellow', hovercolor='0.975')
        self.button_next.on_clicked(self.next_click)
        self.button_prev.on_clicked(self.prev_click)        
        
        # Set text color for the buttons to black
        self.button_next.label.set_color('black')
        self.button_prev.label.set_color('black')
        
    def next_click(self, event):
        if self.current_index < len(self.df.trialNr.unique()) - 1:
            self.current_index += 1
            self.update_plot()
        else:
            self.current_index = self.current_index - len(self.df.trialNr.unique()) # if at the end of df, wrap over to start of df
            self.update_plot()

    def prev_click(self, event):
        if self.current_index > 0:
            self.current_index -= 1
            self.update_plot()
        else:
            self.current_index = self.current_index + len(self.df.trialNr.unique())-1 # if at the start of df, wrap over to end of df
            self.update_plot()

    def update_plot(self):
        self.ax.clear()               
        
        # select a trial based on current_index
        current_trial = self.df.trialNr.unique()[self.current_index]
        df = self.df[self.df.trialNr == current_trial]
        
        if self.plot_type == '2D':        
            # plot 2D graph       
            self.plot2D(df, current_trial = current_trial)
        else:        
            # plot 1D graph
            self.plot1D(df, current_trial = current_trial)       

        
    def plot2D(self, df, current_trial):
        

        # plot raw samples
        self.ax.scatter(df['user_pred_px_x'], df['user_pred_px_y'], c='orange', alpha=0.5, edgecolors='black', label='raw samples')

        # plot fixations and remove no fixations/saccades (zeros)
        self.ax.scatter(df['FixXPos'][df['FixXPos']>0], df['FixYPos'][df['FixYPos']>0], c='blue', alpha=0.5, edgecolors='black', label='fixations')
        
#         # plot target and fixation cirle
#         self.ax.scatter(df.fixationStimX, df.fixationStimY, c='red')
#         self.ax.scatter(df.targetX, df.fixationStimY, c='green')
        
#         # plot target and fixation vertical lines
#         self.ax.plot(np.ones(df.resY.iloc[0].astype('int')) * df.fixationStimX.iloc[0], np.arange(df.resY.iloc[0]), c='red', lw=1, linestyle='dashed')
#         self.ax.plot(np.ones(df.resY.iloc[0].astype('int')) * df.targetX.iloc[0], np.arange(df.resY.iloc[0]), c='green', lw=1, linestyle='dashed')

        # set axis limits based on screen resolution
        self.ax.set_xlim((0, df.resX.iloc[0]))
        self.ax.set_ylim((df.resY.iloc[0]), 0)
        
        # label the axes
        self.ax.set_xlabel('Horizontal eye position (pixels)')
        self.ax.set_ylabel('Vertical eye position (pixels)')
        
        self.ax.legend()
        self.ax.set_title(f'Target {df.locStudiedImage.iloc[0]}, Trial {current_trial}')    
        self.ax.grid(True)
        
        plt.draw()
        
   

    def plot1D(self, df, current_trial):  
                     
        # rescale time variable to start from zero
        t = np.array(df.sampTime)
        t = t-t[0]
        
               
        # plot raw points and fixations
        self.ax.scatter(t, df.user_pred_px_x, c='orange', alpha=0.5, edgecolors='black', label='raw_samples')
        self.ax.scatter(t, df.FixXPos, c='blue', alpha=0.5, edgecolors='black', label='fixations')
        
        # add target and fixaton horizontal lines
        self.ax.plot(t, df.targetX, c='green', lw=1)
        self.ax.plot(t, df.fixationStimX, c='red', lw=1, linestyle='dashed')
        
        # label the axes
        self.ax.set_xlabel('Time (ms)')
        self.ax.set_ylabel('Horizontal eye position (pixels)')
        
        # set axis limits based on horizantal resolution
        self.ax.set_ylim((df.resX.iloc[0]), 0)
        
        # plot saccade latency as a horizontal line
        self.ax.plot(np.array([df.SaccLat.iloc[0]] * df.resX.iloc[0].astype(int)), np.arange(df.resX.iloc[0]), c='black', lw=1)
        
        
        # set the rest
        self.ax.set_title(f'Target {df.target.iloc[0]}, Trial {current_trial}')
        self.ax.legend()
        self.ax.grid(True)
        
        plt.draw()    

In [40]:
def makeHeat(screenRes, xPos, yPos):
        xMax = screenRes[0]
        yMax = screenRes[1]
        xMin = 0
        yMin = 0
        kernelPar = 50

        # Input handeling
        xlim = np.logical_and(xPos < xMax, xPos > xMin)
        ylim = np.logical_and(yPos < yMax, yPos > yMin)
        xyLim = np.logical_and(xlim, ylim)
        dataX = xPos[xyLim]
        dataX = np.floor(dataX)
        dataY = yPos[xyLim]
        dataY = np.floor(dataY)

        # initiate map and gauskernel
        gazeMap = np.zeros([int((xMax-xMin)),int((yMax-yMin))])+0.0001
        gausKernel = krn.Gaussian2DKernel(kernelPar)

        # Rescale the position vectors (if xmin or ymin != 0)
        dataX -= xMin
        dataY -= yMin

        # Now extract all the unique positions and number of samples
        xy = np.vstack((dataX, dataY)).T
        uniqueXY, idx, counts = uniqueRows(xy)
        uniqueXY = uniqueXY.astype(int)
        # populate the gazeMap
        gazeMap[uniqueXY[:,0], uniqueXY[:,1]] = counts

        # Convolve the gaze with the gauskernel
        heatMap = np.transpose(krn.convolve_fft(gazeMap,gausKernel))
        heatMap = heatMap/np.max(heatMap)

        return heatMap

def uniqueRows(x):
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, idx, counts = np.unique(y, return_index=True, return_counts = True)
    uniques = x[idx]
    return uniques, idx, counts


def np_euclidean_distance(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.sum(np.square(y_pred - y_true), axis=-1))



# Preprocess, extract fixations and add them to the dataframe

In [6]:
sys.path.append('./FixationDetection')
from I2MC import runI2MC

# Path to data folders
path_to_folders = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/pilot_data/'
# path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer/pilot'

# get all folder names
folder_names = os.listdir(path_to_folders)

# read and process original datafile for each participant
for fn in folder_names:
    path = os.path.join(path_to_folders, fn, fn+'_record.csv')       
        
    df = pd.read_csv(path)        

# order frames and drop duplicate samples (with same sampleTime)
df = df[df.fName.notna()]
df.frameNr = df.frameNr.apply(pd.to_numeric, errors='coerce') # if framerNr is not a number, it is replaces with nan
df = df[df.frameNr.notna()] # filter out rows where frameNr is a nan

df = df[df.sampTime.notna()]
df = df[df.user_pred_px_x.notna()]
df = df[df.user_pred_px_y.notna()]
df = df.apply(pd.to_numeric, errors='ignore') # if str convert str to numbers

df = df.sort_values('frameNr')
df = df.reset_index(drop=True)
# df = df.drop_duplicates(subset=['user_pred_px_x', 'user_pred_px_y'], ignore_index=True)
df = df.drop_duplicates(subset=['sampTime'], ignore_index=True)

# get fixations for the original datafile for each participant
fixDF = runI2MC(path, plotData = False)

# add extracted fixations to the original data file (two new columns)
# for each timestamp where fixation was detected, FixXPos and FixYPos are added
idx = 0 # index of fixDF
FixXPos = np.zeros(df.shape[0])
FixYPos = np.zeros(df.shape[0])
FixStartEnd = np.empty(df.shape[0], dtype='U10')
FixStartEnd.fill('') # explicitly fill the array (good practice)
FixDur = np.zeros(df.shape[0])

DistFromPrevFix = np.zeros(df.shape[0])
PrevFixXPos = np.zeros(df.shape[0])
PrevFixYPos = np.zeros(df.shape[0])
prev_fix_x = False # keep track of xy when fixation ends
prev_fix_y = False

PrevFixSampTime = np.zeros(df.shape[0])
prev_fix_sampTime = 0

# iterate thru the original dataframe, thru each sample
for index, row in df.iterrows():
    
    # make sure not to iterate out of range
    if idx < fixDF.shape[0]:
        
        # go to next fixation when fixation ends
        if row['sampTime'] > np.array(fixDF.FixEnd)[idx]:
                idx += 1
        
        # make sure not to iterate out of range
        if idx < fixDF.shape[0]:
            
            # when samples are within fixation, accumulate FixXPos and FixYPos
            if row['sampTime'] >= np.array(fixDF.FixStart)[idx] and row['sampTime'] <= np.array(fixDF.FixEnd)[idx]:

                FixXPos[index] = (np.array(fixDF.XPos)[idx])
                FixYPos[index] = (np.array(fixDF.YPos)[idx])
            
            # label samples on which fixation starts and ends
            if row['sampTime'] == np.array(fixDF.FixStart)[idx]:             
                FixStartEnd[index] = 'fix_start'
                
                if prev_fix_x != False:
                    
                    PrevFixXPos[index] = prev_fix_x
                    PrevFixYPos[index] = prev_fix_y
                    
                    DistFromPrevFix[index] = np.sqrt((np.array(fixDF.XPos)[idx] - prev_fix_x)**2 
                                            + (np.array(fixDF.YPos)[idx] - prev_fix_y)**2)
                    PrevFixSampTime[index] = prev_fix_sampTime
                    
            
            elif row['sampTime'] == np.array(fixDF.FixEnd)[idx]:                
                FixStartEnd[index] = 'fix_end' 
                FixDur[index] = np.array(fixDF.FixDur)[idx]
                                    
                prev_fix_x = np.array(fixDF.XPos)[idx]
                prev_fix_y = np.array(fixDF.YPos)[idx]
                prev_fix_sampTime = np.array(row['sampTime'])
                
       

 
# add fixations to original dataframe
df['FixXPos'] = np.array(FixXPos)
df['FixYPos'] = np.array(FixYPos)
df['FixStartEnd'] = FixStartEnd
df['FixDur'] = np.array(FixDur)
df['DistFromPrevFix'] = DistFromPrevFix
df['PrevFixSampTime'] = PrevFixSampTime
df['PrevFixXPos'] = PrevFixXPos
df['PrevFixYPos'] = PrevFixYPos

# Save the pre-processed dataframe
df.to_csv(os.path.join(path_to_folders, fn, fn+'_record_extra.csv'), index=False)  


# Extract only samples when the target was presented
df = df[df.event=='target_on']

# # Label trials with too few data points
# a = df.groupby('trialNr').count().reset_index()
# a = a[['trialNr', 'sampTime']]
# # 3) rename the columns so they would be added
# a.columns = ['trialNr', 'samplesPerTrial']
# df = pd.merge(df, a, on="trialNr")





Importing and processing: "C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/pilot_data/2023_10_18_18_58_57\2023_10_18_18_58_57_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 2.6466188430786133s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


# Get and plot saccade latency distribution

In [42]:
# Get saccade latency

def getSaccLat(df):    

    # Get timestamp of when target was presented and add it to the dataframe

    # 1) get the first time sample when the target is presented
    sampTime_df = df.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

    # 2)extract the columns needed
    sampTime_df = sampTime_df[['trialNr', 'sampTime']]

    # 3) rename the columns so they would be added
    sampTime_df.columns = ['trialNr', 'targSampTime']

    # 4) merge the target time into the main df (one time per trial)
    df = pd.merge(df, sampTime_df, on="trialNr")

    # Extract saccade latencies

    # 1) select only rows where fixation started
    sl_df = df[df.FixStartEnd == 'fix_start']

    # 2) select only rows with large enough preceeding saccade
    sl_df = sl_df[sl_df.DistFromPrevFix > 300]

    # 3) compute first fixation duration (saccade latency)
    sl_df['SaccLat'] = sl_df.PrevFixSampTime - sl_df.targSampTime

    # 4) remove rows where negative Saccade Latencies for trials where no fixation end is present
    sl_df = sl_df[sl_df.SaccLat > 0]
    
    return sl_df

# 5) plot saccade latencies
sl_df = getSaccLat(df)

plt.figure()
plt.style.use('ggplot')
plt.hist(sl_df.SaccLat)
plt.xlabel('Saccade latency (ms)')

# 6) merge saccade latencies into main df
sl_df = sl_df[['trialNr', 'SaccLat']]
df = pd.merge(df, sl_df, on="trialNr")

# save figure
# plt.savefig('./Figures_MullerLyer/saccade_latency.jpg', dpi=1000, pad_inches=0)


# 1D Plot to compare raw samples to fixations

### 1. Plot separately for target left and target right
### 2. while looping through each trial
    a. rescale time variable to start from 0
    b. plot all raw x
    c. plot all fixationXPos
    d. zeros indicate saccade/no fixation

In [43]:

conditions = ['right', 'left']


for cond in conditions:
    
    plt.figure()
    plt.style.use('ggplot')
    plt.title(f'Target {cond}')
    
    a = df[df.target==cond]

    for name, group in a.groupby('trialNr'):
        t = np.array(group.sampTime)
        t = t-t[0]
        raw_h = plt.scatter(t, group.user_pred_px_x, c='orange', alpha=0.5, edgecolors='black')
        fix_h = plt.scatter(t, group.FixXPos, c='blue', alpha=0.5, edgecolors='black')

    plt.plot(t, group.targetX, c='green', lw=1)
    plt.plot(t, group.fixationStimX, c='red', lw=1, linestyle='dashed')
    
    plt.xlabel('Time (ms)')
    plt.ylabel('Horizontal eye position (pixels)')
    
    plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)
    
    # save figure
#     plt.savefig(f'./Figures_MullerLyer/1D_{cond}.jpg', dpi=1000, pad_inches=0)


# Interactive 1D plot

In [56]:
# Create a figure and initialize the plot
fig, ax = plt.subplots(figsize=(8, 6))
plot_updater = PlotUpdater(ax, df, plot_type='1D')
plot_updater.update_plot()

# Position the 'Next' and 'Previous' buttons at the bottom of the plot
button_ax_prev = plt.axes([0.6, 0.01, 0.1, 0.05])
button_ax_next = plt.axes([0.8, 0.01, 0.1, 0.05])

plot_updater.button_next.ax = button_ax_next
plot_updater.button_prev.ax = button_ax_prev

plt.show()

# 2D plot of fixations and raw samples

## 1. plot all raw x,y
## 2. plot all fixations x,y 


In [8]:
import matplotlib.patches as patches

conditions = ['right', 'left', 'both']


for cond in conditions:
    
    plt.figure()
    plt.style.use('ggplot')
    plt.title(f'Studied image {cond}')
        
    # select condition
    a = df[df.locStudiedImage==cond]  
               
    raw_h = plt.scatter(a.user_pred_px_x, a.user_pred_px_y, c='orange', alpha=0.5, edgecolors='black')
    
    # remove no fixations/saccades (zeros)
    fix_h = plt.scatter(a.FixXPos[a.FixXPos>0], a.FixYPos[a.FixYPos>0], c='blue', alpha=0.5, edgecolors='black') 
    
    # plot center fixation dot
    plt.scatter(a.resX.iloc[0]/2, a.resY.iloc[0]/2, c='red')
    
    # plot image boxes   
    left = a.leftX.iloc[0]
    right = a.rightX.iloc[0]
    top = a.Y.iloc[0]
    width = 480
    height = 480
    left_rect=patches.Rectangle((left,top),width,height, 
                        fill = False,
                        color = "purple",
                        linewidth = 2)
    
    right_rect=patches.Rectangle((right,top),width,height, 
                        fill = False,
                        color = "purple",
                        linewidth = 2)
    
    plt.gca().add_patch(left_rect)
    plt.gca().add_patch(right_rect)
    
    
    plt.xlim((0, df.resX.iloc[0]))
    plt.ylim((df.resY.iloc[0]), 0)
    
    plt.xlabel('Horizontal eye position (pixels)')
    plt.ylabel('Vertical eye position (pixels)')
    
    plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)
    
    # save figure
    plt.savefig(os.path.join(path_to_folders, 'analysis', fn+f'2D{cond}.jpg'), dpi=1000, pad_inches=0)


# Calculate novelty index


In [14]:
# select condition
testPhase_df = df[df.phase=='test']
# testPhase_df =  testPhase_df.drop_duplicates(subset=['FixXPos', 'FixYPos'], ignore_index=True) # one unique fixation per row
testPhase_df = testPhase_df[testPhase_df.FixStartEnd=='fix_end']

screen_centerX = testPhase_df.resX.iloc[0]/2
distFixToImageBoarder = testPhase_df.distBetweenImages.iloc[0]/2 - 480/2

# filter out fixations not reaching the image box
testPhase_df = testPhase_df[((testPhase_df.FixXPos < (screen_centerX-distFixToImageBoarder))
                                     | (testPhase_df.FixXPos > (screen_centerX+distFixToImageBoarder)))]

# label the fixation on left or right side                                     
testPhase_df['FixatedImage'] = np.where(testPhase_df.FixXPos < screen_centerX, 'left', 'right')
testPhase_df['FixatedNovel'] = np.where(testPhase_df.FixatedImage == testPhase_df.locStudiedImage, 'old', 'novel')


novelty_idx_fix = []
novelty_idx_fixDur = []
# iterate through trials
for i, group in testPhase_df.groupby('trialNr'):
        
    # calculate the proportion of novel fixations on each trial
    novelty_idx_fix.append(group.FixatedNovel[group.FixatedNovel=='novel'].count() / group.FixatedNovel.count())
    
    # calculate the proportion of novel fixationTime on each trial
    novelty_idx_fixDur.append(group.FixDur[group.FixatedNovel=='novel'].sum() / group.FixDur.sum())

novelty_idx_fix = np.array(novelty_idx_fix)
novelty_idx_fixDur = np.array(novelty_idx_fixDur)

# Prepare outout df
output_df = testPhase_df.drop_duplicates(subset=['trialNr'], ignore_index=True) # one trial per row
output_df = output_df.drop(['frameNr','sampTime', 'user_pred_px_x', 'user_pred_px_y'], axis=1) # drop columns by name
output_df = output_df.iloc[:,:20] # drop columns by index

# Log the novelty indices per trial
output_df['noveltyIdx_fixCountProp'] = novelty_idx_fix
output_df['noveltyIdx_fixDurProp'] = novelty_idx_fixDur

# Save the output file
# output_df.to_csv(os.path.join(path_to_folders, 'analysis', fn+'_analyzed.csv'))

# Interactive 2D plot for every trial

In [11]:
# Create a figure and initialize the plot
fig, ax = plt.subplots(figsize=(8, 6))
plot_updater = PlotUpdater(ax, df, plot_type='2D')
plot_updater.update_plot()

# Position the 'Next' and 'Previous' buttons at the bottom of the plot
button_ax_prev = plt.axes([0.6, 0.01, 0.1, 0.05])
button_ax_next = plt.axes([0.8, 0.01, 0.1, 0.05])

plot_updater.button_next.ax = button_ax_next
plot_updater.button_prev.ax = button_ax_prev

plt.show()